<a href="https://colab.research.google.com/github/adalves-ufabc/2022.Q2-PLN/blob/main/2022_Q2_PLN_Notebook_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2022.Q2]**
Prof. Alexandre Donizeti Alves

### **TF-IDF**

---

In [1]:
# carregando os quatro documentos
doc1 = "cachorro morde homem"
doc2 = "homem morde cachorro"
doc3 = "cachorro come carne"
doc4 = "homem come comida"

In [2]:
# separando por espacos
bow1 = doc1.split(" ")
bow2 = doc2.split(" ")
bow3 = doc3.split(" ")
bow4 = doc4.split(" ")

In [3]:
bow1

['cachorro', 'morde', 'homem']

In [4]:
# unindo os documentos em um corpus
wordSet = set(bow1).union(set(bow2)).union(set(bow3).union(set(bow4)))
wordSet

{'cachorro', 'carne', 'come', 'comida', 'homem', 'morde'}

In [5]:
# criando dicionarios de palavras, frequencias
wordDict1 = dict.fromkeys(wordSet, 0) 
wordDict2 = dict.fromkeys(wordSet, 0)
wordDict3 = dict.fromkeys(wordSet, 0)
wordDict4 = dict.fromkeys(wordSet, 0)

In [6]:
wordDict1

{'cachorro': 0, 'carne': 0, 'come': 0, 'comida': 0, 'homem': 0, 'morde': 0}

In [7]:
# incrementando a frequencia de cada dict
for word in bow1:
    wordDict1[word]+=1
    
for word in bow2:
    wordDict2[word]+=1
    
for word in bow3:
    wordDict3[word]+=1

for word in bow4:
    wordDict4[word]+=1

In [8]:
wordDict1

{'cachorro': 1, 'carne': 0, 'come': 0, 'comida': 0, 'homem': 1, 'morde': 1}

In [9]:
import pandas as pd

pd.DataFrame([wordDict1, wordDict2, wordDict3, wordDict4])

,come,comida,morde,cachorro,homem,carne
0,0,0,1,1,1,0
1,0,0,1,1,1,0
2,1,0,0,1,0,1
3,1,1,0,0,1,0


In [10]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [11]:
# computando TF
tfBow1 = computeTF(wordDict1, bow1)
tfBow2 = computeTF(wordDict2, bow2)
tfBow3 = computeTF(wordDict3, bow3)
tfBow4 = computeTF(wordDict4, bow4)

In [ ]:
tfBow1

{'cachorro': 0.3333333333333333,
 'carne': 0.0,
 'come': 0.0,
 'comida': 0.0,
 'homem': 0.3333333333333333,
 'morde': 0.3333333333333333}

In [12]:
def termFrequency(term, document):
    normalizeDocument = document.lower().split()
    return normalizeDocument.count(term.lower()) / float(len(normalizeDocument))

In [13]:
termFrequency('cachorro', doc1)

0.3333333333333333

In [14]:
# computando IDF
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    # idfDict = {'come': 2, 'homem': 3, 'cachorro': 3, 'carne': 1, 'comida': 1, 'morde': 2}
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))

    return idfDict

In [15]:
idfs = computeIDF([wordDict1, wordDict2, wordDict3, wordDict4])

In [16]:
# computando TF-IDF
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [17]:
tfidfBow1 = computeTFIDF(tfBow1, idfs)
tfidfBow2 = computeTFIDF(tfBow2, idfs)
tfidfBow3 = computeTFIDF(tfBow3, idfs)
tfidfBow4 = computeTFIDF(tfBow4, idfs)

In [18]:
import pandas as pd

pd.DataFrame([tfidfBow1, tfidfBow2, tfidfBow3, tfidfBow4])

,come,comida,morde,cachorro,homem,carne
0,0.000000,0.000000,0.100343,0.041646,0.041646,0.000000
1,0.000000,0.000000,0.100343,0.041646,0.041646,0.000000
2,0.100343,0.000000,0.000000,0.041646,0.000000,0.200687
3,0.100343,0.200687,0.000000,0.000000,0.041646,0.000000


**Importante**: quanto maior o valor do TF-IDF, mais raro é o termo. Quanto menor o TF-IDF, mais comum é o termo.

### **Usando a biblioteca `Scikit-Learn`**

"Scikit-Learn" é uma biblioteca de aprendizado de máquina de código aberto para a linguagem de programação Python.

https://scikit-learn.org/stable/

In [20]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([doc1, doc2, doc3, doc4])

feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
pd.DataFrame(denselist, columns=feature_names)

,cachorro,carne,come,comida,homem,morde
0,0.53257,0.000000,0.000000,0.000000,0.53257,0.657829
1,0.53257,0.000000,0.000000,0.000000,0.53257,0.657829
2,0.44810,0.702035,0.553492,0.000000,0.00000,0.000000
3,0.00000,0.000000,0.553492,0.702035,0.44810,0.000000


Existem 2 maneiras diferentes de implementar o **TF-IDF** usando o `Scikit-Learn`. Uma é usando a classe `TfidfVectorizer` (como acabamos de fazer) e a outra é usando a classe `TfidfTransformer`. Você pode ter se perguntado qual é a diferença entre as 2, então vamos discutir isso.

Teoricamente falando, não há diferença entre as 2 implementações. Na prática, precisamos escrever mais algum código se quisermos usar o `TfidfTransformer`. A principal diferença entre as 2 implementações é que o `TfidfVectorizer` calcula a frequência do termo (TF) e o inverso da frequência nos documentos (IDF) para você, enquanto o uso do `TfidfTransformer` exigirá que você use a classe `CountVectorizer` do `Scikit-Learn` para calcular a frequência do termo.

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer

dataset = [
    "I enjoy reading about Machine Learning and Machine Learning is my PhD subject",
    "I would enjoy a walk in the park",
    "I was reading in the library"]

In [23]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(dataset)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

print (df.head(10))

            TF-IDF
machine   0.513720
learning  0.513720
about     0.256860
subject   0.256860
phd       0.256860
and       0.256860
my        0.256860
is        0.256860
reading   0.195349
enjoy     0.195349


In [24]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

tfIdfTransformer = TfidfTransformer(use_idf=True)
countVectorizer = CountVectorizer()
wordCount = countVectorizer.fit_transform(dataset)
newTfIdf = tfIdfTransformer.fit_transform(wordCount)
df = pd.DataFrame(newTfIdf[0].T.todense(), index=countVectorizer.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

print (df.head(10))

            TF-IDF
machine   0.513720
learning  0.513720
about     0.256860
subject   0.256860
phd       0.256860
and       0.256860
my        0.256860
is        0.256860
reading   0.195349
enjoy     0.195349


Portanto, TF-IDF é uma ponderação (*score*) que é aplicada a cada palavra em cada documento em nosso conjunto de dados. E para cada palavra, o valor TF-IDF aumenta com cada aparição da palavra em um documento, mas diminui gradualmente com cada aparição em outros documentos.

### **Aplicações de TF-IDF**

Portanto, vimos que implementar TF-IDF usando as ferramentas certas é muito fácil, mas as aplicações desse algoritmo são muito poderosas. Os dois casos de uso mais comuns para TF-IDF são:

> **Recuperação de informações**: calculando a ponderação TF-IDF de uma busca do usuário em relação a todo o conjunto de documentos, podemos descobrir a relevância de um documento para aquela consulta. Há rumores de que a maioria dos mecanismos de busca usa algum tipo de implementação TF-IDF.

> **Extração de palavras-chave**: as palavras de classificação mais alta para um documento em termos de ponderação TF-IDF podem muito bem representar as palavras-chave desse documento (pois fazem com que esse documento se destaque dos outros documentos). 